In [1]:
 #!pip install imutils

In [2]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import dlib

import math
import imutils

In [3]:
#사진 불러오기
my_image_path = './images/img1.jpg'
img_bgr = cv2.imread(my_image_path)    # OpenCV로 이미지를 불러옵니다
img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
plt.imshow(img_rgb)
plt.show()

## cv2.imread(img_path, flag)
이미지 파일을 flag값에 따라서 읽어들입니다.
- cv2.IMREAD_COLOR : 이미지 파일을 Color로 읽어들입니다. 투명한 부분은 무시되며, Default값입니다.
- cv2.IMREAD_GRAYSCALE : 이미지를 Grayscale로 읽어 들입니다. 실제 이미지 처리시 중간단계로 많이 사용합니다.
- cv2.IMREAD_UNCHANGED : 이미지파일을 alpha channel까지 포함하여 읽어 들입니다.

3개의 flag대신에 1, 0, -1을 사용해도 됩니다.

In [4]:
img_show = img_bgr.copy()      # 출력용 이미지를 따로 보관합니다

In [5]:
#얼굴 검출 face detection
detector_hog = dlib.get_frontal_face_detector()

dlib_rects = detector_hog(img_rgb, 1)   # (image, num of image pyramid)

for dlib_rect in dlib_rects:
    l = dlib_rect.left()
    t = dlib_rect.top()
    r = dlib_rect.right()
    b = dlib_rect.bottom()

    cv2.rectangle(img_show, (l,t), (r,b), (0,255,0), 2, lineType=cv2.LINE_AA)

plt.imshow(cv2.cvtColor(img_show, cv2.COLOR_BGR2RGB))
plt.show()

In [6]:
#모델 불러오기
model_path = './models/shape_predictor_68_face_landmarks.dat'
landmark_predictor = dlib.shape_predictor(model_path)

In [7]:
list_landmarks = []

# 얼굴 영역 박스 마다 face landmark를 찾아냅니다
for dlib_rect in dlib_rects:
    points = landmark_predictor(img_rgb, dlib_rect)
    # face landmark 좌표를 저장해둡니다
    list_points = list(map(lambda p: (p.x, p.y), points.parts()))
    list_landmarks.append(list_points)
    
for landmark in list_landmarks:
    for point in landmark:
        cv2.circle(img_show, point, 7, (0, 0, 255), -1)

plt.imshow(cv2.cvtColor(img_show, cv2.COLOR_BGR2RGB))
plt.show()

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/E-8-8.png)
- Dlib landmark localization

- 30과 27의 좌표를 위 식에 대입하여 회전할 각도를 계산합니다

In [8]:
#스티커 위치 구하기
for dlib_rect, landmark in zip(dlib_rects, list_landmarks):
    x = landmark[30][0]
    y = landmark[30][1] 
    w = h = dlib_rect.width()
    x2 = landmark[27][0]
    y2 = landmark[27][1]
    
    tan_theta = (x-x2)/(y-y2)
    theta = np.arctan(tan_theta)
    rotate_angle = -theta *180/math.pi

    print ('(x,y) : (%d,%d)'%(x,y))
    print ('(x2,y2) : (%d,%d)'%(x2,y2))
    print ('(w,h) : (%d,%d)'%(w,h))
    print ('degree: ',rotate_angle)

(x,y) : (710,558)
(x2,y2) : (706,462)
(w,h) : (386,386)
degree:  -2.3859440303888126


In [9]:
#스티커 적용
sticker_path = './images/cat-whiskers.png'
img_sticker = cv2.imread(sticker_path) # 스티커 이미지를 불러옵니다
img_sticker = cv2.resize(img_sticker, (w,h//2))
img_sticker = imutils.rotate_bound(img_sticker, rotate_angle)  

print (img_sticker.shape)
plt.imshow(img_sticker)
plt.show()

(208, 393, 3)


- `imutils.rotate_bound()`를 사용하여 이미지를 잘림없이 회전시켜줍니다.

In [10]:
refined_x = x - img_sticker.shape[1]//2
refined_y = y - img_sticker.shape[0]//2

#음수 좌표 처리
if refined_x < 0: 
    img_sticker = img_sticker[:, -refined_x:]
    refined_x = 0
if refined_y < 0:
    img_sticker = img_sticker[-refined_y:, :]
    refined_y = 0

print ('(x,y) : (%d,%d)'%(refined_x, refined_y))

(x,y) : (514,454)


In [11]:
#스티커 적용
# 길어서 복잡해 보이지만 img_show[from:to] 형식입니다
sticker_area = img_show[refined_y:refined_y+img_sticker.shape[0], refined_x:refined_x+img_sticker.shape[1]]
img_show[refined_y:refined_y+img_sticker.shape[0], 
         refined_x:refined_x+img_sticker.shape[1]] = \
         np.where(img_sticker==0,sticker_area,img_sticker).astype(np.uint8)
                #이미지에 따라 img_sticker==255로 적용해야 될 수도 있습니다.
    
#결과 확인
plt.imshow(cv2.cvtColor(img_show, cv2.COLOR_BGR2RGB))
plt.show()

## np.where(조건, True, False)
- 조건식에 만족하는 pixel은 True값으로 치환, 조건을 만족하지 못하는 pixel은 False에 기입된 값으로 치환하는 함수입니다.

In [12]:
## 결과 출력
sticker_area = img_rgb[refined_y:refined_y +img_sticker.shape[0], refined_x:refined_x+img_sticker.shape[1]]
sticker_area = cv2.cvtColor(sticker_area, cv2.COLOR_RGB2BGR)

img_bgr[refined_y:refined_y+img_sticker.shape[0], 
         refined_x:refined_x+img_sticker.shape[1]] = \
         np.where(img_sticker==0,sticker_area,img_sticker).astype(np.uint8)

result = cv2.addWeighted(img_rgb, 0.3, cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB), 0.7, 0.0)
plt.imshow(result)
plt.show()

## cv2.addWeighted(src1, alpha, src2, beta, gamma, dst=None, dtype=None) -> dst

- src1: (입력) 첫 번째 이미지
- alpha: 첫 번째 이미지 가중치
- src2: 두 번째 이미지. src1과 같은 크기 & 같은 타입
- beta: 두 번째 이미지 가중치
- gamma: 결과 이미지에 추가적으로 더할 값
- dst: 가중치 합 결과 이미지 
- dtype: 출력 영상(dst)의 타입

`cv2.add()`도 있는데 이를 사용할 경우 값을 더하기만하여 이미지의 결과가 너무 밝아 질 수 있어 weight값을 줄 수 있는 `cv2.addWeighted()`를 사용하였습니다.

---
# 결과


- img2와 img3에서 각도에 따라 고양이 수염의 모습이 잘 바뀌었습니다.
- img3에서 인물의 거리가 멀어졌음에도 잘 인식하였습니다.
- 어두운 환경에서 찍은 img4도 마찬가지로 잘 적용되었습니다.

---
# 회고

이번 프로젝트를 진행하면서 opencv의 기본적인 사용법에 대하여 알게되었습니다.
스티커이미지의 크기와 각도를 변경하고 좌표계산을 통하여 적절한 위치에 반영하였습니다.
opencv에서는 이미지를 rgb가아닌 bgr로 읽어들이는데 이를 염두에 두고 코딩해야된다는 생각이 들었습니다.